In [1]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime
from datetime import timedelta

from coordenadas_a_cluster import *

modelo = load_model("kmeans.21.3.joblib")

/home/sjn/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
#!ls -l
#!rm 300107-0-agenda-actividades-eventos.csv
#!ls -l
#!wget https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.csv
#!ls -l

In [3]:
#eventos_df = pd.read_csv('300107-0-agenda-actividades-eventos.csv', encoding = 'ISO-8859-1', sep=';', engine='python')
#eventos_df.head(5)

In [4]:

url_agenda_eventos = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.csv"

response = requests.get(url_agenda_eventos, headers={"User-Agent": "curl/7.61.0"})

data = StringIO(response.content.decode('ISO-8859-1'))

eventos_df = pd.read_csv(data, sep=';', engine='python')
eventos_df.head(5)



,ID-EVENTO,TITULO,PRECIO,GRATUITO,LARGA-DURACION,DIAS-SEMANA,DIAS-EXCLUIDOS,FECHA,FECHA-FIN,HORA,...,URL-INSTALACION,NOMBRE-INSTALACION,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,DISTRITO,TIPO,AUDIENCIA,Unnamed: 23
0,11070336,1815: el origen de una Revolución. Miserias (1...,NaN,1,0,NaN,NaN,2019-12-28 00:00:00.0,2019-12-28 23:59:00.0,16:30,...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Valdebernardo (Vicálvaro),447278.0,4472321.0,40.399829,-3.621282,VICALVARO,/contenido/actividades/TeatroPerformance,NaN,NaN
1,11070337,1815: el origen de una Revolución. Miserias (1...,NaN,1,0,NaN,NaN,2019-12-29 00:00:00.0,2019-12-29 23:59:00.0,11:00,...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Valdebernardo (Vicálvaro),447278.0,4472321.0,40.399829,-3.621282,VICALVARO,/contenido/actividades/TeatroPerformance,NaN,NaN
2,11070338,1815: el origen de una Revolución. Miserias (1...,NaN,1,0,NaN,NaN,2019-12-29 00:00:00.0,2019-12-29 23:59:00.0,16:30,...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Valdebernardo (Vicálvaro),447278.0,4472321.0,40.399829,-3.621282,VICALVARO,/contenido/actividades/TeatroPerformance,NaN,NaN
3,11070320,1815: el origen de una Revolución. Miserias (1...,NaN,1,0,NaN,NaN,2019-12-20 00:00:00.0,2019-12-20 23:59:00.0,18:00,...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Valdebernardo (Vicálvaro),447278.0,4472321.0,40.399829,-3.621282,VICALVARO,/contenido/actividades/TeatroPerformance,NaN,NaN
4,11070323,1815: el origen de una Revolución. Miserias (2...,NaN,1,0,NaN,NaN,2019-12-21 00:00:00.0,2019-12-21 23:59:00.0,11:00,...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Valdebernardo (Vicálvaro),447278.0,4472321.0,40.399829,-3.621282,VICALVARO,/contenido/actividades/TeatroPerformance,NaN,NaN


In [5]:
print(eventos_df.shape)
eventos_reduced_df = eventos_df[['TITULO', 'GRATUITO', 'DIAS-SEMANA', 'FECHA', 'FECHA-FIN', 'HORA', 'LATITUD', 'LONGITUD']]

(1000, 24)


In [6]:

eventos_reduced_df.tail(5)


,TITULO,GRATUITO,DIAS-SEMANA,FECHA,FECHA-FIN,HORA,LATITUD,LONGITUD
995,Talleres de iniciación a la Ornitología: Dehes...,1,NaN,2019-11-23 00:00:00.0,2019-11-23 23:59:00.0,10:30,40.45818,-3.720725
996,Talleres de iniciación a la Ornitología: Parqu...,1,NaN,2019-12-07 00:00:00.0,2019-12-07 23:59:00.0,10:30,NaN,NaN
997,Talleres de manualidades para niños y niñas de...,1,"S,D",2019-11-02 00:00:00.0,2019-12-01 23:59:00.0,11:30,NaN,NaN
998,Talleres de reciclaje. Abónate: haz tu propio ...,1,NaN,2019-12-22 00:00:00.0,2019-12-22 23:59:00.0,11:30,40.41790,-3.730939
999,Talleres de reciclaje. Balcón comestible.Casa ...,1,NaN,2019-11-24 00:00:00.0,2019-11-24 23:59:00.0,11:30,40.41790,-3.730939


In [7]:
print('Limpiamos el dataset de elementos que no tengan los campos obligatorios')
eventos_reduced_df = eventos_reduced_df.dropna(subset=['HORA', 'LATITUD', 'LONGITUD', 'FECHA', 'FECHA-FIN'])

eventos_reduced_df = eventos_reduced_df.drop(eventos_reduced_df[eventos_reduced_df['HORA'] == '00:00'].index)
eventos_reduced_df.tail(10)


Limpiamos el dataset de elementos que no tengan los campos obligatorios


,TITULO,GRATUITO,DIAS-SEMANA,FECHA,FECHA-FIN,HORA,LATITUD,LONGITUD
986,Taller 'Me cuido para cuidar' en el Centro de ...,1,NaN,2019-11-28 00:00:00.0,2019-11-28 23:59:00.0,11:00,40.380566,-3.765724
987,Taller 'Navidad verde en El Retiro',1,NaN,2019-12-14 00:00:00.0,2019-12-14 23:59:00.0,11:00,40.419641,-3.687934
988,Talleres de anillamiento científico. Casa de C...,1,NaN,2019-12-15 00:00:00.0,2019-12-15 23:59:00.0,09:30,40.414724,-3.730186
989,Talleres de anillamiento científico. Casa de C...,1,NaN,2019-11-30 00:00:00.0,2019-11-30 23:59:00.0,09:30,40.414724,-3.730186
990,Talleres de anillamiento científico. Dehesa de...,1,NaN,2019-12-01 00:00:00.0,2019-12-01 23:59:00.0,09:30,40.459141,-3.718271
993,Talleres de Bici en Dehesa de la Villa,1,NaN,2019-12-22 00:00:00.0,2019-12-22 23:59:00.0,11:00,40.458180,-3.720725
994,Talleres de iniciación a la Ornitología: Dehes...,1,NaN,2019-12-14 00:00:00.0,2019-12-14 23:59:00.0,10:30,40.458180,-3.720725
995,Talleres de iniciación a la Ornitología: Dehes...,1,NaN,2019-11-23 00:00:00.0,2019-11-23 23:59:00.0,10:30,40.458180,-3.720725
998,Talleres de reciclaje. Abónate: haz tu propio ...,1,NaN,2019-12-22 00:00:00.0,2019-12-22 23:59:00.0,11:30,40.417900,-3.730939
999,Talleres de reciclaje. Balcón comestible.Casa ...,1,NaN,2019-11-24 00:00:00.0,2019-11-24 23:59:00.0,11:30,40.417900,-3.730939


In [8]:
print(eventos_reduced_df.shape)

(660, 8)


In [9]:
eventos_reduced_df['FECHA'] = pd.to_datetime(eventos_reduced_df['FECHA'])
eventos_reduced_df['FECHA-FIN'] = pd.to_datetime(eventos_reduced_df['FECHA-FIN'])

In [10]:

dias_semana = ['L', 'M', 'X', 'J', 'V', 'S', 'D']

eventos_final = []

for _, row in eventos_reduced_df.iterrows():
    
    fecha = row['FECHA']
    fecha_fin = row['FECHA-FIN']
    fecha_day = row['FECHA'].day
    fecha_month = row['FECHA'].month
    fecha_year = row['FECHA'].year
    fecha_fin_day = row['FECHA-FIN'].day
    fecha_fin_month = row['FECHA-FIN'].month
    fecha_fin_year = row['FECHA-FIN'].year
    dias_semana_evento = row['DIAS-SEMANA']
    
    hora_evento = row['HORA'].split(':')    
    
    # Transformar cada evento con su timestamp de cuándo comienza el evento
    fecha = fecha.replace(hour=int(hora_evento[0]), minute=int(hora_evento[1]), second=0, microsecond=0)

    print('fecha_evento:\t\t' + str(fecha))
    print('fecha_fin_evento:\t' + str(fecha_fin))
    print('dias_semana_evento:\t' + str(dias_semana_evento))
    print('\n')

    if (fecha_day != fecha_fin_day or fecha_month != fecha_fin_month or fecha_year != fecha_fin_year):

        # Generamos tantos eventos como días toquen según la programación de 'DIAS-SEMANA'
        while fecha < fecha_fin:

            dia_semana = dias_semana[fecha.weekday()]
            
            # Si el día tiene evento, lo generamos y guardamos en el array
            if (dia_semana in dias_semana_evento):
                #print('este día hay evento\n')
                evento_nuevo = {}
            
                evento_nuevo['TITULO'] = row['TITULO']
                evento_nuevo['GRATUITO'] = row['GRATUITO']
                evento_nuevo['FECHA'] = fecha
                evento_nuevo['FECHA-FIN'] = row['FECHA-FIN']
                evento_nuevo['HORA'] = row['HORA']
                evento_nuevo['LATITUD'] = row['LATITUD']
                evento_nuevo['LONGITUD'] = row['LONGITUD']
                evento_nuevo['CLUSTER'] = coordenadas_a_cluster(row['LONGITUD'], row['LATITUD'], modelo)
                
                eventos_final.append(evento_nuevo)

            fecha = fecha + timedelta(days = 1)
    else:
        evento_nuevo = {}
        
        evento_nuevo['TITULO'] = row['TITULO']
        evento_nuevo['GRATUITO'] = row['GRATUITO']
        evento_nuevo['FECHA'] = fecha
        evento_nuevo['FECHA-FIN'] = row['FECHA-FIN']
        evento_nuevo['HORA'] = row['HORA']
        evento_nuevo['LATITUD'] = row['LATITUD']
        evento_nuevo['LONGITUD'] = row['LONGITUD']
        evento_nuevo['CLUSTER'] = coordenadas_a_cluster(row['LONGITUD'], row['LATITUD'], modelo)

        eventos_final.append(evento_nuevo)


eventos_final_df = pd.DataFrame(eventos_final)
    
    

fecha_evento:		2019-12-28 16:30:00
fecha_fin_evento:	2019-12-28 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-29 11:00:00
fecha_fin_evento:	2019-12-29 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-29 16:30:00
fecha_fin_evento:	2019-12-29 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-20 18:00:00
fecha_fin_evento:	2019-12-20 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-21 11:00:00
fecha_fin_evento:	2019-12-21 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-21 16:30:00
fecha_fin_evento:	2019-12-21 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-22 11:00:00
fecha_fin_evento:	2019-12-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-22 16:30:00
fecha_fin_evento:	2019-12-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-23 18:00:00
fecha_fin_evento:	2019-12-23 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-26 18:00:00
fecha_fin_evento:	2019-12-26 23:59:00
dias_semana_evento:	nan


fecha_even

fecha_evento:		2019-12-03 18:00:00
fecha_fin_evento:	2019-12-03 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-05 19:00:00
fecha_fin_evento:	2019-12-05 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-30 20:00:00
fecha_fin_evento:	2019-11-30 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-13 18:00:00
fecha_fin_evento:	2019-12-13 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-06 11:00:00
fecha_fin_evento:	2019-11-27 23:59:00
dias_semana_evento:	L,M,X,J,V,S,D


fecha_evento:		2019-12-04 11:00:00
fecha_fin_evento:	2019-12-18 23:59:00
dias_semana_evento:	X


fecha_evento:		2019-10-30 19:30:00
fecha_fin_evento:	2020-03-19 23:59:00
dias_semana_evento:	L,M,X,J,V,S,D


fecha_evento:		2019-11-30 18:30:00
fecha_fin_evento:	2019-11-30 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-12 18:00:00
fecha_fin_evento:	2019-12-12 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-25 18:00:00
fecha_fin_evento:	2019-11-25 23:59:00
dias_semana_evento

fecha_evento:		2019-11-07 18:30:00
fecha_fin_evento:	2019-11-28 23:59:00
dias_semana_evento:	J


fecha_evento:		2019-10-31 19:00:00
fecha_fin_evento:	2019-12-12 23:59:00
dias_semana_evento:	J


fecha_evento:		2019-11-22 15:30:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-14 15:30:00
fecha_fin_evento:	2019-12-14 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-15 15:30:00
fecha_fin_evento:	2019-12-15 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-20 19:00:00
fecha_fin_evento:	2019-12-20 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-22 19:00:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-26 20:00:00
fecha_fin_evento:	2019-11-26 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-22 19:30:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-14 19:00:00
fecha_fin_evento:	2019-12-14 23:59:00
dias_semana_evento:	nan


fecha_evento:	

fecha_evento:		2019-12-14 18:00:00
fecha_fin_evento:	2019-12-14 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-15 12:00:00
fecha_fin_evento:	2019-12-15 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-10-28 10:00:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	L,M,X,J,V,S,D


fecha_evento:		2019-11-27 19:00:00
fecha_fin_evento:	2019-11-27 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-26 18:00:00
fecha_fin_evento:	2019-11-26 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-26 12:30:00
fecha_fin_evento:	2019-12-26 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-25 19:00:00
fecha_fin_evento:	2019-11-25 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-15 13:00:00
fecha_fin_evento:	2020-04-30 23:59:00
dias_semana_evento:	L,M,X,J,V,S,D


fecha_evento:		2019-11-11 17:00:00
fecha_fin_evento:	2019-12-03 23:59:00
dias_semana_evento:	L,M,X,J,V,S


fecha_evento:		2019-11-23 18:30:00
fecha_fin_evento:	2019-11-23 23:59:00
dias_sem

fecha_evento:		2019-11-02 18:00:00
fecha_fin_evento:	2019-11-30 23:59:00
dias_semana_evento:	V


fecha_evento:		2019-11-24 18:00:00
fecha_fin_evento:	2019-11-24 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-24 17:00:00
fecha_fin_evento:	2019-11-24 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-29 19:00:00
fecha_fin_evento:	2019-11-29 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-23 18:30:00
fecha_fin_evento:	2019-11-23 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-24 12:00:00
fecha_fin_evento:	2019-11-24 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-22 18:00:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-29 18:30:00
fecha_fin_evento:	2019-11-29 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-05 19:00:00
fecha_fin_evento:	2019-12-05 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-29 20:30:00
fecha_fin_evento:	2019-12-14 23:59:00
dias_semana_evento:	V,S


fecha_evento

fecha_evento:		2019-11-22 19:00:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-27 17:00:00
fecha_fin_evento:	2019-11-27 23:59:00
dias_semana_evento:	nan


fecha_evento:		2018-12-12 13:00:00
fecha_fin_evento:	2019-12-31 23:59:00
dias_semana_evento:	L,M,X,J,V,S,D


fecha_evento:		2019-12-08 12:00:00
fecha_fin_evento:	2019-12-15 23:59:00
dias_semana_evento:	D


fecha_evento:		2019-11-28 19:00:00
fecha_fin_evento:	2019-11-28 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-16 16:00:00
fecha_fin_evento:	2019-11-30 23:59:00
dias_semana_evento:	L,M,X,J,V,S,D


fecha_evento:		2019-11-22 19:30:00
fecha_fin_evento:	2019-11-22 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-12-14 11:30:00
fecha_fin_evento:	2019-12-14 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-26 18:30:00
fecha_fin_evento:	2019-11-26 23:59:00
dias_semana_evento:	nan


fecha_evento:		2019-11-24 12:30:00
fecha_fin_evento:	2019-11-24 23:59:00
dias_semana_evento

In [11]:
eventos_final_df.shape

(2755, 8)

In [12]:
eventos_final_df.head(5)

,CLUSTER,FECHA,FECHA-FIN,GRATUITO,HORA,LATITUD,LONGITUD,TITULO
0,10,2019-12-28 16:30:00,2019-12-28 23:59:00,1,16:30,40.399829,-3.621282,1815: el origen de una Revolución. Miserias (1...
1,10,2019-12-29 11:00:00,2019-12-29 23:59:00,1,11:00,40.399829,-3.621282,1815: el origen de una Revolución. Miserias (1...
2,10,2019-12-29 16:30:00,2019-12-29 23:59:00,1,16:30,40.399829,-3.621282,1815: el origen de una Revolución. Miserias (1...
3,10,2019-12-20 18:00:00,2019-12-20 23:59:00,1,18:00,40.399829,-3.621282,1815: el origen de una Revolución. Miserias (1...
4,10,2019-12-21 11:00:00,2019-12-21 23:59:00,1,11:00,40.399829,-3.621282,1815: el origen de una Revolución. Miserias (2...


In [13]:
eventos_final_df.tail(5)

,CLUSTER,FECHA,FECHA-FIN,GRATUITO,HORA,LATITUD,LONGITUD,TITULO
2750,140,2019-12-22 11:00:00,2019-12-22 23:59:00,1,11:00,40.45818,-3.720725,Talleres de Bici en Dehesa de la Villa
2751,140,2019-12-14 10:30:00,2019-12-14 23:59:00,1,10:30,40.45818,-3.720725,Talleres de iniciación a la Ornitología: Dehes...
2752,140,2019-11-23 10:30:00,2019-11-23 23:59:00,1,10:30,40.45818,-3.720725,Talleres de iniciación a la Ornitología: Dehes...
2753,93,2019-12-22 11:30:00,2019-12-22 23:59:00,1,11:30,40.41790,-3.730939,Talleres de reciclaje. Abónate: haz tu propio ...
2754,93,2019-11-24 11:30:00,2019-11-24 23:59:00,1,11:30,40.41790,-3.730939,Talleres de reciclaje. Balcón comestible.Casa ...


In [14]:
#pip install mysql-connector-python

import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
from config import *


In [17]:

if __name__ == "__main__":
    
    nuevos = 0
    existentes = 0
    
    try:
        connection = mysql.connector.connect(
            host=db_host,
            user=db_user,
            passwd=db_passwd,
            database=db_database,
            port=db_port
        )

        cursor = connection.cursor()
        
        for _, row in eventos_final_df.iterrows():
            #print(row)
            sql = f'INSERT INTO DatosEventos(fecha, gratuito, titulo, longitud, latitud, cluster) \
            VALUES \
            (\'{row.get("FECHA")}\', {row.get("GRATUITO")}, \"{row.get("TITULO")}\", {row.get("LONGITUD")}, {row.get("LATITUD")}, {row.get("CLUSTER")});'
            #print(sql)
            try:
                cursor.execute(sql)
                nuevos = nuevos + 1
            except mysql.connector.Error as err:
                existentes = existentes + 1
                print("Error: {}".format(err))

        connection.commit()
        print("Records inserted successfully into DatosEventos table")
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into DatosEventos table {}".format(error))

    finally:
        if (connection.is_connected()):
            connection.close()
            print("MySQL connection is closed")
    print("\tnuevos: " + str(nuevos))
    print("\texistentes: " + str(existentes))
    text = "Proceso de Carga de Eventos finalizado. \n\tNuevos: {0}\n\tTotal: {1}".format(str(nuevos), str(existentes))
    payload = {'chat_id': tg_group, 'text': text}
    requests.post(tg_url, data=payload)
print("-----------\n")


Error: 1062 (23000): Duplicate entry '2019-12-28 16:30:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-29 11:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-29 16:30:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-20 18:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-21 11:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-21 16:30:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-22 11:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-22 16:30:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-23 18:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-26 18:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-27 18:00:00-10' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-

Error: 1062 (23000): Duplicate entry '2020-04-20 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-04-27 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-05-04 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-05-11 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-05-18 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-05-25 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-06-01 20:00:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-03 18:00:00-11' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-05 19:00:00-107' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-30 20:00:00-194' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-13 18:00:00-44' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '201

Error: 1062 (23000): Duplicate entry '2019-11-29 11:00:00-4' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-10-02 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-10-09 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-10-16 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-10-23 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-10-30 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-06 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-13 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-20 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-27 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-04 18:00:00-145' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate ent

Error: 1062 (23000): Duplicate entry '2019-12-07 19:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-14 12:30:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-13 19:30:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-26 17:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-07 11:00:00-119' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-30 11:00:00-92' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-30 12:30:00-92' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-01 11:00:00-34' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-28 11:00:00-34' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-15 11:00:00-34' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-24 17:30:00-92' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019

Error: 1062 (23000): Duplicate entry '2020-03-13 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-14 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-15 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-16 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-17 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-18 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-19 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-20 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-21 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-22 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-03-23 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-

...

Error: 1062 (23000): Duplicate entry '2019-12-10 18:00:00-100' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-23 12:00:00-100' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-22 19:00:00-14' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-30 19:00:00-102' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-10 19:00:00-149' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-01-30 18:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-01-31 18:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-02-01 18:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-02-02 18:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-29 19:30:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-30 12:30:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2

Error: 1062 (23000): Duplicate entry '2019-03-21 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-22 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-23 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-24 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-25 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-26 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-27 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-28 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-29 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-30 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-03-31 13:00:00-46' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-

Error: 1062 (23000): Duplicate entry '2019-11-22 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-23 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-24 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-25 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-26 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-27 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-28 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-29 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-30 16:00:00-24' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-22 19:30:00-42' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-12-14 11:30:00-93' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-

Error: 1062 (23000): Duplicate entry '2020-10-20 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-10-27 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-11-03 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-11-10 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-11-17 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-11-24 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-12-01 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-12-08 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2020-12-15 17:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-21 16:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-11-28 16:30:00-54' for key 'eve_unique_idx'
Error: 1062 (23000): Duplicate entry '2019-

-----------

